In [63]:
import libpysal
import numpy
import geopandas
import matplotlib
import matplotlib.pyplot as plt
import spaghetti
import shapely
from shapely.geometry import Point

In [64]:
def simulated_geo_points(in_data, needed=20, seed=0, to_file=None):
    """Generate synthetic spatial data points within an area.s
    Parameters
    ----------
    in_data : geopandas.GeoDataFrame
        A single polygon of the unioned street buffers.
    needed : int
        Number of points in the buffer. Default is 20.
    seed : int
        Seed for pseudo-random number generation. Default is 0.
    to_file : str
        File name for write out.
    Returns
    -------
    sim_pts : geopandas.GeoDataFrame
        Points within the buffer.
    """
    geoms = in_data.geometry
    area = tuple(in_data.total_bounds)
    simulated_points_list = []
    simulated_points_all = False
    numpy.random.seed(seed)
    while simulated_points_all == False:
        x = numpy.random.uniform(area[0], area[2], 1)
        y = numpy.random.uniform(area[1], area[3], 1)
        point = Point(x, y)
        if geoms.intersects(point)[0]:
            simulated_points_list.append(point)
        if len(simulated_points_list) == needed:
            simulated_points_all = True
    sim_pts = geopandas.GeoDataFrame(
        simulated_points_list, columns=["geometry"], crs=in_data.crs
    )
    if to_file:
        sim_pts.to_file(to_file + ".shp")
    return sim_pts

In [87]:
lattice = spaghetti.regular_lattice((0,0,10,10), 9, exterior=True)
ntw = spaghetti.Network(in_data=lattice)
gdf = spaghetti.element_as_gdf(ntw, arcs=True)
street = geopandas.GeoDataFrame(geopandas.GeoSeries(gdf['geometry'].buffer(0.2).unary_union), crs=gdf.crs, columns=['geometry'])

In [125]:
client_count = 100
facility_count = 5

client_points = simulated_geo_points(street, needed=client_count, seed=5)
facility_points = simulated_geo_points(street, needed=facility_count, seed=6)

In [113]:
display(facility_points)
display(client_points)

geometry
0  POINT (9.08575 3.25259)

geometry
0    POINT (2.10873 8.85562)
1    POINT (1.94988 9.35355)
2    POINT (4.87948 6.16214)
3    POINT (7.76544 5.19155)
4    POINT (2.88673 1.75230)
..                       ...
95  POINT (-0.13128 4.30248)
96   POINT (5.86437 3.42781)
97   POINT (2.20274 0.07079)
98  POINT (7.40431 10.18456)
99   POINT (0.19876 8.73462)

[100 rows x 1 columns]

In [126]:
## Cost Matrix
ntw = spaghetti.Network(in_data=lattice)

ntw.snapobservations(client_points, "clients", attribute=True)
clients_snapped = spaghetti.element_as_gdf(
    ntw, pp_name="clients", snapped=True
)
ntw.snapobservations(facility_points, "facilities", attribute=True)
facilities_snapped = spaghetti.element_as_gdf(
    ntw, pp_name="facilities", snapped=True
)

cost_matrix = ntw.allneighbordistances(
    sourcepattern=ntw.pointpatterns["clients"],
    destpattern=ntw.pointpatterns["facilities"],
)
cost_matrix

array([[12.60302601,  3.93598651,  8.16571655,  6.04319467,  5.65607701],
       [13.10096347,  4.43392397,  8.66365401,  6.54113213,  5.15813955],
       [ 6.9095462 ,  4.2425067 ,  2.47223674,  0.34971486,  5.34955682],
       [ 2.98196832,  7.84581224,  3.45534114,  3.57786302,  6.25374871],
       [ 7.5002892 ,  6.32806975,  4.55779979,  6.43527791, 11.75939222],
       [ 0.60209077, 11.42987132,  5.03940023,  7.16192211,  9.8378078 ],
       [ 5.37335867,  6.20113923,  2.43086927,  4.30834738,  9.6324617 ],
       [ 5.40801577,  5.41976478,  3.02929369,  1.15181557,  4.85108725],
       [ 3.68807115,  8.51585171,  2.12538061,  4.24790249,  7.94717417],
       [14.22503627,  4.60274429,  9.78772681,  7.66520493,  4.98931924],
       [10.32521229,  4.99225179,  7.38272288,  9.260201  , 14.58431531],
       [ 6.65436171,  7.98732222,  5.59685112,  3.719373  ,  2.58135531],
       [11.55510375,  1.11193575,  7.11779429,  5.37988496, 10.70399927],
       [10.90832519,  1.75871431,  6.4

In [132]:
from spopt.locate.coverage import LSCP
from pulp import GLPK
lscp = LSCP.from_cost_matrix(cost_matrix, max_coverage=8)
lscp.solve(GLPK())
#%

1

In [140]:
from spopt.locate.coverage import MCLP
from pulp import GLPK
ai = numpy.random.randint(1, 12, (client_count, 1))
mclp = MCLP.from_cost_matrix(cost_matrix, ai, max_coverage=4, p_facilities=2,)
mclp.solve(GLPK())

1